# Quanta Maths: Integer Addition and Subtraction in Transformers. Scan 200+ LLMs

This Colab uses the app.withmartian.com API to test 200+ models
to see whether given query like "Answer concisely: 4444+5559=" they answer "10003".

In [1]:
# Non-obvious cascading carry tests
# Units column ≥10, all other columns sum to exactly 9
add_tests = [
    ['Answer concisely: 6+5=', '11'], # score 1
    ['Answer concisely: 19+87=', '106'], # score 2
    ['Answer concisely: 774+229=', '1003'], # score 3
    ['Answer concisely: 6587+3416=', '10003'], # score 4
    ['Answer concisely: 22605+77398=', '100003'], # score 5
    ['Answer concisely: 532847+467159=', '1000006'], # score 6
    ['Answer concisely: 5613709+4386294=', '10000003'], # score 7
    ['Answer concisely: 72582383+27417619=', '100000002'], # score 8
    ['Answer concisely: 206727644+793272359=', '1000000003'], # score 9
    ['Answer concisely: 7580116456+2419883549=', '10000000005'], # score 10
    ['Answer concisely: 52449010267+47550989737=', '100000000004'], # score 11
    ['Answer concisely: 888522030597+111477969406=', '1000000000003'], # score 12
    ['Answer concisely: 3764410205738+6235589794265=', '10000000000003'], # score 13
    ['Answer concisely: 45928371046529+54071628953474=', '100000000000003'], # score 14
    ['Answer concisely: 712850394621847+287149605378157=', '1000000000000004'], # score 15
]

# Cascading borrow tests
# Smaller number minus larger number, requires cascading borrows, results in negative answers
sub_tests = [
    ['Answer concisely: 23-47=', '-24'], # score 1
    ['Answer concisely: 156-289=', '-133'], # score 2
    ['Answer concisely: 2047-5183=', '-3136'], # score 3
    ['Answer concisely: 13056-27394=', '-14338'], # score 4
    ['Answer concisely: 240157-583269=', '-343112'], # score 5
    ['Answer concisely: 1350268-2794736=', '-1444468'], # score 6
    ['Answer concisely: 24601379-51836472=', '-27235093'], # score 7
    ['Answer concisely: 135702468-269485731=', '-133783263'], # score 8
    ['Answer concisely: 2461035789-5138274692=', '-2677238903'], # score 9
    ['Answer concisely: 13570246801-27394851639=', '-13824604838'], # score 10
    ['Answer concisely: 246103578924-518372469157=', '-272268890233'], # score 11
    ['Answer concisely: 1357024680135-2739485163728=', '-1382460483593'], # score 12
    ['Answer concisely: 24610357892461-51837246915839=', '-27226889023378'], # score 13
    ['Answer concisely: 135702468013570-273948516372946=', '-138246048359376'], # score 14
    ['Answer concisely: 2461035789246103-5183724691583727=', '-2722688902337624'], # score 15
]

max_score_possible = 15

In [2]:
import re 

def is_ground_truth_correct(answer, ground_truth):
    """
    Returns True if the ground_truth appears as the final number in the answer, ignoring whitespace and punctuation.
    Accepts answers like '13', '13.', '13**', 'The answer is 13', '**13**', 'random text **13** random text', 'boxed{13}'.
    """
    # Remove trailing whitespace and punctuation
    answer_clean = answer.strip().rstrip('.!**')
    # Find all numbers in the answer (including negative numbers and those with commas)
    numbers = re.findall(r'-?[\d,]+', answer_clean)
    
    # Remove commas from the numbers for comparison
    numbers_clean = [num.replace(',', '') for num in numbers]

    answer_no_comma = answer.replace(",", "")

    return (ground_truth == answer_no_comma or
            "**"+ground_truth+"**" in answer or
            "boxed{"+ground_truth+"}" in answer or
            ""+ground_truth+" " in answer_no_comma  or
            ""+ground_truth+"." in answer_no_comma  or
            (numbers_clean and numbers_clean[-1] == ground_truth))

In [3]:
assert is_ground_truth_correct("-14338", "-14338")
assert is_ground_truth_correct("-14338 ", "-14338")
assert is_ground_truth_correct("-14338.", "-14338")
assert is_ground_truth_correct("-14,338", "-14338")
assert is_ground_truth_correct("-14,338 ", "-14338")
assert is_ground_truth_correct("-14,338.", "-14338")
assert not is_ground_truth_correct("-14339", "-14338")
assert is_ground_truth_correct("boxed{-14338}", "-14338")
assert is_ground_truth_correct("**-14338**", "-14338")
assert is_ground_truth_correct("blah blah-14338 blah blah", "-14338")
assert is_ground_truth_correct("135,702,468 - 269,485,731 = **-133,783,263**", "-133783263")
assert is_ground_truth_correct("12123 - 12312 = -14338", "-14338")

## Martian LLMs

Supported martian models are at https://app.withmartian.com/docs/index.html
and https://api.withmartian.com/v1/models

In [4]:
import requests

# Fetch Martian model data from the API
martian_models_url = "https://api.withmartian.com/v1/models"
response = requests.get(martian_models_url)
martian_models_json = response.json()

# Parse the data into a list of tuples: (id, prompt, completion, image, reliability_tier)
martian_models_data = []
for model in martian_models_json["data"]:
    pricing = model.get("pricing", {})
    martian_models_data.append(
        (
            model.get("id"),
            float(pricing.get("prompt", 0)),
            float(pricing.get("completion", 0)),
            float(pricing.get("image", 0)),
            model.get("reliability_tier")
        )
    )

# martian_models_data now contains the latest data from the endpoint
print(f"Loaded {len(martian_models_data)} models from Martian API.")


Loaded 522 models from Martian API.


In [5]:
import re
from typing import List, Tuple, Optional, Dict, Any

In [6]:
# Helper functions for data analysis

def extract_model_names() -> List[str]:
    """Extract just the model names from the data"""
    return [model[0] for model in martian_models_data]

def extract_param_size(param_str: Optional[str]) -> Optional[float]:
    """Convert parameter string to float (in billions)"""
    if param_str is None:
        return None
    # If already a number, just return as float
    if isinstance(param_str, (int, float)):
        return float(param_str)
    param_str = str(param_str)
    # Remove 'B' and convert to float
    if param_str.endswith('B'):
        param_str = param_str[:-1]
    # Handle mixtral format like "8x22B" -> approximate total params
    if 'x' in param_str:
        parts = param_str.split('x')
        try:
            return float(parts[0]) * float(parts[1])
        except Exception:
            return None
    try:
        return float(param_str)
    except Exception:
        return None

def get_models_by_provider() -> Dict[str, List[Tuple[str, float, float, float, Optional[str]]]]:
    """Group models by provider"""
    providers = {}
    for model_data in martian_models_data:
        provider = model_data[0].split('/')[0]
        if provider not in providers:
            providers[provider] = []
        providers[provider].append(model_data)
    return providers

def get_cost_stats() -> Dict[str, Any]:
    """Get cost statistics"""
    input_costs = [model[1] for model in martian_models_data]
    output_costs = [model[2] for model in martian_models_data]
    return {
        'input_cost_range': (min(input_costs), max(input_costs)),
        'output_cost_range': (min(output_costs), max(output_costs)),
        'avg_input_cost': sum(input_costs) / len(input_costs),
        'avg_output_cost': sum(output_costs) / len(output_costs)
    }

def get_param_stats() -> Dict[str, Any]:
    """Get parameter count statistics"""
    params_with_data = [extract_param_size(model[4]) for model in martian_models_data if model[4] is not None]
    params_with_data = [p for p in params_with_data if p is not None]
    if not params_with_data:
        return {'count': 0}
    return {
        'count': len(params_with_data),
        'range': (min(params_with_data), max(params_with_data)),
        'avg_params': sum(params_with_data) / len(params_with_data),
        'models_with_param_info': len([m for m in martian_models_data if m[4] is not None]),
        'total_models': len(martian_models_data)
    }

def find_cheapest_models(top_n: int = 5) -> List[Tuple[str, float, float]]:
    """Find the cheapest models by input cost"""
    sorted_by_input = sorted(martian_models_data, key=lambda x: x[1])
    return [(model[0], model[1], model[2]) for model in sorted_by_input[:top_n]]

def find_most_expensive_models(top_n: int = 5) -> List[Tuple[str, float, float]]:
    """Find the most expensive models by input cost"""
    sorted_by_input = sorted(martian_models_data, key=lambda x: x[1], reverse=True)
    return [(model[0], model[1], model[2]) for model in sorted_by_input[:top_n]]

def find_largest_models(top_n: int = 10) -> List[Tuple[str, Optional[str], float]]:
    """Find the largest models by parameter count"""
    models_with_params = [(model[0], model[4], extract_param_size(model[4])) 
                         for model in martian_models_data if model[4] is not None]
    models_with_params = [(m[0], m[1], m[2]) for m in models_with_params if m[2] is not None]
    sorted_by_params = sorted(models_with_params, key=lambda x: x[2], reverse=True)
    return sorted_by_params[:top_n]


In [7]:
martian_models = extract_model_names()

# Print comprehensive statistics

print("=== MARTIAN AI MODELS ANALYSIS ===\n")

print(f"📊 Total models: {len(martian_models_data)}")

# Provider breakdown
providers = get_models_by_provider()
print(f"🏢 Number of providers: {len(providers)}")
#print("\n📈 Models per provider:")
#for provider, models in sorted(providers.items(), key=lambda x: len(x[1]), reverse=True):
#    print(f"  {provider:20} {len(models):3d} models")

# Cost analysis
cost_stats = get_cost_stats()
print(f"\n💰 Cost Analysis (per 1M tokens):")
print(f"  Input cost range:  ${cost_stats['input_cost_range'][0]:.2f} - ${cost_stats['input_cost_range'][1]:.2f}")
print(f"  Output cost range: ${cost_stats['output_cost_range'][0]:.2f} - ${cost_stats['output_cost_range'][1]:.2f}")
print(f"  Average input:     ${cost_stats['avg_input_cost']:.2f}")
print(f"  Average output:    ${cost_stats['avg_output_cost']:.2f}")

# Parameter analysis
param_stats = get_param_stats()
print(f"\n🧠 Parameter Analysis:")
print(f"  Models with param info: {param_stats['models_with_param_info']}/{param_stats['total_models']}")
if param_stats['count'] > 0:
    print(f"  Parameter range: {param_stats['range'][0]:.1f}B - {param_stats['range'][1]:.1f}B")
    print(f"  Average size: {param_stats['avg_params']:.1f}B parameters")

# Top lists
print(f"\n💸 5 Cheapest Models (input cost):")
for i, (model, input_cost, output_cost) in enumerate(find_cheapest_models(), 1):
    print(f"  {i}. {model:40} ${input_cost:.2f}/${output_cost:.2f}")

print(f"\n💎 5 Most Expensive Models (input cost):")
for i, (model, input_cost, output_cost) in enumerate(find_most_expensive_models(), 1):
    print(f"  {i}. {model:40} ${input_cost:.2f}/${output_cost:.2f}")

print(f"\n🦣 10 Largest Models:")
for i, (model, param_str, param_float) in enumerate(find_largest_models(), 1):
    print(f"  {i:2d}. {model:40} {param_str:>8} ({param_float:.1f}B)")

print(f"\n📋 Sample model data structure:")
print(f"   Format: (model_name, input_cost_per_1M, output_cost_per_1M, request_cost, params)")
for i in range(3):
    model = martian_models_data[i]
    print(f"   {model}")

print(f"\n✅ Use 'martian_models' for just the model names list")
print(f"✅ Use 'martian_models_data' for full data with costs and parameters")

=== MARTIAN AI MODELS ANALYSIS ===

📊 Total models: 522
🏢 Number of providers: 43

💰 Cost Analysis (per 1M tokens):
  Input cost range:  $0.00 - $0.00
  Output cost range: $0.00 - $0.00
  Average input:     $0.00
  Average output:    $0.00

🧠 Parameter Analysis:
  Models with param info: 522/522
  Parameter range: 0.0B - 2.0B
  Average size: 1.5B parameters

💸 5 Cheapest Models (input cost):
  1. meta-llama/llama-3.2-1b-instruct:cheap   $0.00/$0.00
  2. liquid/lfm-7b:cheap                      $0.00/$0.00
  3. meta-llama/llama-3.2-1b-instruct         $0.00/$0.00
  4. liquid/lfm-3b:cheap                      $0.00/$0.00
  5. liquid/lfm-7b                            $0.00/$0.00

💎 5 Most Expensive Models (input cost):
  1. openai/o1-pro                            $0.00/$0.00
  2. openai/o1-pro:cheap                      $0.00/$0.00
  3. openai/gpt-4                             $0.00/$0.00
  4. anthropic/claude-3-opus-20240229         $0.00/$0.00
  5. anthropic/claude-opus-4-0            

## Run Models

In [8]:
import os
from dotenv import load_dotenv
import openai
import concurrent.futures
import time
import re
from concurrent.futures import ThreadPoolExecutor, as_completed

In [9]:
load_dotenv()
MARTIAN_API_KEY = os.getenv("MARTIAN_API_KEY")
assert MARTIAN_API_KEY, "API key not found. Please set MARTIAN_API_KEY in your .env file."

In [10]:
client = openai.OpenAI(
    base_url="https://api.withmartian.com/v1",
    api_key=MARTIAN_API_KEY
)

In [11]:
def run_model_inference(model_name, prompt, ground_truth, timeout=300):

    response = client.chat.completions.create(
        model=model_name,
        max_tokens=1024, # Mandatory param for some models. Ignored by others.
        messages=[{"role": "user", "content": prompt}]
    )
    answer = response.choices[0].message.content.strip()
    success = is_ground_truth_correct(answer, ground_truth)
    return answer, success

In [12]:
def evaluate_models_progressive(the_models, tests, max_workers=8):
    model_scores = []

    def score_model(model_name):
        score = 0
        for test_idx, (prompt, ground_truth) in enumerate(tests):
            try:
                answer, success = run_model_inference(model_name, prompt, ground_truth)
                if success:
                    score = test_idx + 1
                else:
                    break
            except openai.APIError as e:
                if hasattr(e, 'status_code'):
                    score = -e.status_code
                else:
                    score = -999
                break
            except Exception as e:
                score = -999
                break
        return {"model": model_name, "score": score}

    print(f"Evaluating {len(the_models)} models concurrently with {max_workers} workers...")
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_model = {executor.submit(score_model, model_name): model_name for model_name in the_models}
        for idx, future in enumerate(as_completed(future_to_model), 1):
            model_name = future_to_model[future]
            try:
                result = future.result()
            except Exception as exc:
                result = {"model": model_name, "score": -999}
            print(f"[{idx}/{len(the_models)}] {result['model']}: Score = {result['score']}")
            model_scores.append(result)
    return model_scores

In [ ]:
# Run the tasks across all models except those containing ':cheap'. This tasks ~45 minutes on a laptop.
the_models = [model[0] for model in martian_models_data if ':cheap' not in model[0]]
add_model_scores = evaluate_models_progressive(the_models, add_tests)
sub_model_scores = evaluate_models_progressive(the_models, sub_tests)

Evaluating 261 models concurrently with 8 workers...
[1/261] alfredpros/codellama-7b-instruct-solidity: Score = 0
[1/261] alfredpros/codellama-7b-instruct-solidity: Score = 0
[2/261] allenai/molmo-7b-d: Score = 3
[2/261] allenai/molmo-7b-d: Score = 3
[3/261] aion-labs/aion-rp-llama-3.1-8b: Score = 2
[3/261] aion-labs/aion-rp-llama-3.1-8b: Score = 2
[4/261] allenai/olmo-2-0325-32b-instruct: Score = 4
[4/261] allenai/olmo-2-0325-32b-instruct: Score = 4
[5/261] ai21/jamba-mini-1.7: Score = 4
[6/261] amazon/nova-micro-v1: Score = 3
[7/261] alpindale/goliath-120b: Score = 4
[5/261] ai21/jamba-mini-1.7: Score = 4
[6/261] amazon/nova-micro-v1: Score = 3
[7/261] alpindale/goliath-120b: Score = 4
[8/261] anthracite-org/magnum-v4-72b: Score = 4
[8/261] anthracite-org/magnum-v4-72b: Score = 4
[9/261] ai21/jamba-large-1.7: Score = 8
[9/261] ai21/jamba-large-1.7: Score = 8
[10/261] anthropic/claude-3-5-haiku-20241022: Score = 6
[10/261] anthropic/claude-3-5-haiku-20241022: Score = 6
[11/261] amazon

# Summarize results

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
 
def plot_model_scores(model_scores, title, score_threshold):
    # Summarize number of models by score
    print(title)
    print(f"Total number of models: {len(martian_models)}")
    score_counts = Counter([m['score'] for m in model_scores])
    print("Score summary (number of models by score):")
    for score, count in sorted(score_counts.items()):
        print(f"Score {score}: {count} models") 

    # Print total number of models with negative and non-negative scores
    num_negative = sum(count for score, count in score_counts.items() if isinstance(score, int) and score < 0)
    num_nonneg = sum(count for score, count in score_counts.items() if isinstance(score, int) and score >= 0)
    print(f"\nTotal models with negative score: {num_negative}")
    print(f"Total models with non-negative score: {num_nonneg}")

    # Print percentage of models with non-negative scores that scored score_threshold or more
    num_good = sum(count for score, count in score_counts.items() if isinstance(score, int) and score >= score_threshold)
    if num_nonneg > 0:
        percent_good = 100 * num_good / num_nonneg
        print(f"Percentage of non-negative models scoring {score_threshold} or more: {percent_good:.2f}%")
    else:
        print("No models with non-negative scores.")
    print("\n")

    return score_counts

add_score_counts = plot_model_scores(add_model_scores, "Addition Test Scores", max_score_possible)
sub_score_counts = plot_model_scores(sub_model_scores, "Subtraction Test Scores", max_score_possible)

In [ ]:
# Bar graph for Addition and Subtraction score counts, x-axis always 0 to 15
score_range = list(range(max_score_possible+1))
add_counts = [add_score_counts.get(s, 0) for s in score_range]
sub_counts = [sub_score_counts.get(s, 0) for s in score_range]

plt.figure(figsize=(12, 5))
bars_add = plt.bar([x-0.2 for x in score_range], add_counts, width=0.4, color='skyblue', label='Addition')
bars_sub = plt.bar([x+0.2 for x in score_range], sub_counts, width=0.4, color='salmon', label='Subtraction')

plt.xlabel('Model Score', fontsize=18)
plt.ylabel('Number of Models', fontsize=18)
plt.xticks(score_range, [str(s) for s in score_range], fontsize=16)
plt.yticks(fontsize=16)
plt.legend(fontsize=16)

# Add count labels inside bars only where count > 4, with larger font
for bar, count in zip(bars_add, add_counts):
    if count > 4:
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height()/2, str(count), ha='center', va='center', fontsize=16, color='black')
for bar, count in zip(bars_sub, sub_counts):
    if count > 4:
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height()/2, str(count), ha='center', va='center', fontsize=16, color='black')

plt.tight_layout()
plt.show()

In [ ]:
# Scatter plot: Output Token Cost vs. Model Score for Addition and Subtraction (x-axis always 0 to 15)
import numpy as np
model_name_to_output_cost = {model[0]: model[2] for model in martian_models_data}

# Addition scores
add_model_names = [entry['model'] for entry in add_model_scores]
add_scores = [entry['score'] for entry in add_model_scores]
add_output_costs = [model_name_to_output_cost.get(name, np.nan) for name in add_model_names]
add_filtered = [(score, cost) for score, cost in zip(add_scores, add_output_costs) if isinstance(score, int) and 0 <= score <= max_score_possible]
if add_filtered:
    add_filtered_scores, add_filtered_output_costs = zip(*add_filtered)
else:
    add_filtered_scores, add_filtered_output_costs = [], []

# Subtraction scores
sub_model_names = [entry['model'] for entry in sub_model_scores]
sub_scores = [entry['score'] for entry in sub_model_scores]
sub_output_costs = [model_name_to_output_cost.get(name, np.nan) for name in sub_model_names]
sub_filtered = [(score, cost) for score, cost in zip(sub_scores, sub_output_costs) if isinstance(score, int) and 0 <= score <= max_score_possible]
if sub_filtered:
    sub_filtered_scores, sub_filtered_output_costs = zip(*sub_filtered)
else:
    sub_filtered_scores, sub_filtered_output_costs = [], []

# Prepare categorical x-axis for scores 0 to max_score_possible
score_range = list(range(max_score_possible+1))
score_to_x = {score: i for i, score in enumerate(score_range)}
add_x_vals = [score_to_x[score] for score in add_filtered_scores]
sub_x_vals = [score_to_x[score] for score in sub_filtered_scores]

plt.figure(figsize=(10,6))
plt.scatter(add_x_vals, add_filtered_output_costs, alpha=0.6, color='blue', label='Addition')
plt.scatter(sub_x_vals, sub_filtered_output_costs, alpha=0.6, color='red', label='Subtraction')
plt.xlabel('Model Score', fontsize=18)
plt.ylabel('Model Output Token Cost (USD per 1M tokens)', fontsize=16)
plt.xticks(score_range, [str(s) for s in score_range], fontsize=16)
plt.yticks(fontsize=16)
plt.yscale('log')
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend(fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# List only models with a negative score, zero score or score >= 10 for both Addition and Subtraction
def list_models_by_score(model_scores, label):
    score_to_models = {}
    for entry in model_scores:
        score = entry['score']
        if score <= 0 or score >= 10:
            score_to_models.setdefault(score, []).append(entry['model'])
    print(f"\n{label}:")
    for score in sorted(score_to_models):
        print(f"Models with score {score}:")
        for model in score_to_models[score]:
            print(f"  {model}")

list_models_by_score(add_model_scores, "Addition Models")
list_models_by_score(sub_model_scores, "Subtraction Models")

# Check zero-score model output

On occasion, check the output of the "0 score" models to see if unusual answer formats are hiding correct answers.
If so, is_ground_truth_correct should be updated

In [ ]:
# Re-run models that scored 0 on test[0] and show their outputs
def run_zero_score_models(title, model_scores, tests):
    zero_score_models = [m['model'] for m in model_scores if m['score'] == 0]
    print(f"Re-running {len(zero_score_models)} models that scored 0 on the first test for {title}...")
    outputs = []
    for model_name in zero_score_models:
        try:
            answer, success = run_model_inference(model_name, tests[0][0], tests[0][1])
        except Exception as e:
            answer = f"Error: {str(e)}"
            success = False
        outputs.append({'model': model_name, 'output': answer, 'success': success})
        print(f"Model: {model_name}\nOutput: {answer}\nSuccess: {success}\n{'-'*40}")


#run_zero_score_models("Addition", add_model_scores, add_tests)
#run_zero_score_models("Subtraction", sub_model_scores, sub_tests)

# Check -400 score models

Models can generate -400 scores if a mandatory parameter is missing. This may hide a model that can do addition.
Run the models that give a -400 score and see if the detailed error message shows the root cause.

In [ ]:
# Re-run models that scored -400 and show their outputs and errors
def run_minus_400_score_models(title, model_scores, tests):
    minus_400_models = [m['model'] for m in model_scores if m['score'] == -400]
    print(f"Re-running {len(minus_400_models)} models that scored -400 on the first test for {title}...")
    outputs = []
    for model_name in minus_400_models:
        try:
            answer, success = run_model_inference(model_name, tests[0][0], tests[0][1])
        except Exception as e:
            answer = f"Error: {str(e)}"
            success = False
        outputs.append({'model': model_name, 'output': answer, 'success': success})
        print(f"Model: {model_name}\nOutput/Error: {answer}\nSuccess: {success}\n{'-'*40}")

#run_minus_400_score_models("Addition", add_model_scores, add_tests)
#run_minus_400_score_models("Subtraction", sub_model_scores, sub_tests)

# High Variability Example
The model "alfredpros/codellama-7b-instruct-solidity" has high variability. Sometimes answer is correct

In [ ]:
def run_high_variability_model():
    model_name = "alfredpros/codellama-7b-instruct-solidity"
    for i in range(5):
        answer, success = run_model_inference(model_name, add_tests[0][0], add_tests[0][1])
        print(f"Success: {success}. Output: {answer}")

#run_high_variability_model()

# Ask how the models that score >=14 work

Inconclusive: 

Sometimes the detailed methodology is mathematically incorrect.
Sometimes the outline methodology looks like a sentence from a text book.
The bext valid explanations say the model does addition like a human - line up digit pairs - then sum right digit-pair to-left digit pair calculating carries 
The "concise" setting makes these model reply immediately - unless they have many hidden "thinking tokens" the human approach is not feasible.

Some of these models gpt-4o-mini-search-preview and gpt-4o-search-preview explicitly say they have search/tool access. 
So they can just call a python function to calculate the sum.

In [ ]:
def show_work_for_score_best_models(title, model_scores):
    best_models = [m['model'] for m in model_scores if m['score'] >= 14]
    ground_prompt = "998522030597+1477969406="
    concise_prompt = f"Answer concisely: {ground_prompt}"
    explain_prompt = f"Show your work for {ground_prompt}"
    ground_truth = str(998522030597 + 1477969406)

    print(f"Querying {len(best_models)} models that scored best...")
    for model_name in best_models:
        try:
            answer, _ = run_model_inference(model_name, concise_prompt, ground_truth)
        except Exception as e:
            answer = f"Error: {str(e)}"
        print(f"Model: {model_name}\nOutput: {answer}\n{'-'*40}")

        try:
            answer, _ = run_model_inference(model_name, explain_prompt, ground_truth)
        except Exception as e:
            answer = f"Error: {str(e)}"
        print(f"Model: {model_name}\nOutput: {answer}\n{'-'*40}")

#show_work_for_score_best_models("Addition", add_model_scores)
#show_work_for_score_best_models("Subtraction", sub_model_scores)

# Chat GPT 5
What did the Chat GPT 5 models score?

In [ ]:
# Print scores for all models containing 'gpt-5' in both Addition and Subtraction
def print_gpt5_scores(model_scores, label):
    print(f"\n{label}:")
    for entry in model_scores:
        model = entry['model']
        score = entry['score']
        if 'gpt-5' in model:
            print(f"Model: {model}, Score: {score}")

print_gpt5_scores(add_model_scores, "Addition Scores")
print_gpt5_scores(sub_model_scores, "Subtraction Scores")

# Test some subtraction examples 

In [ ]:
import random
from pprint import pprint

# Filter models that scored more than 0 on subtraction
eligible_models = [entry for entry in sub_model_scores if entry['score'] > 0]

if not eligible_models:
    print("No models scored more than 0 on subtraction.")
else:
    # Pick up to 5 random models
    selected_models = random.sample(eligible_models, min(5, len(eligible_models)))

    for entry in selected_models:
        model = entry['model']
        score = entry['score']
        print(f"\nModel: {model} (Subtraction Score: {score})")
        # Run at the score they succeeded at
        if 1 <= score <= max_score_possible:
            test_idx = score - 1
            test_prompt, test_answer = sub_tests[test_idx]
            print(f"Running at succeeded test (Score {score}): {test_prompt}")
            answer1, success1 = run_model_inference(model, test_prompt, test_answer)
            print(test_answer, success1, answer1)
            # Run at the next harder test they failed at
            if score < max_score_possible:
                next_test_idx = score
                next_test_prompt, next_test_answer = sub_tests[next_test_idx]
                print(f"Running at next harder test (Score {score+1}): {next_test_prompt}")
                answer2, success2 = run_model_inference(model, next_test_prompt, next_test_answer)
                print(next_test_answer, success2, answer2)
        else:
            print("Model did not succeed at any subtraction test in the defined range.")